In [2]:
!pip install findspark

In [1]:
# spark 환경 잡음 
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext 

In [3]:
# driver 생성
# pypeJ 가서 jvm 돌고 객체생성해서 가져와야함
sc = SparkContext()

In [4]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [6]:
# driver die 
sc.stop()

---

In [9]:
# getOrCreate() : driver 없으면 생성하고 있으면 불러오고 
sc = SparkContext.getOrCreate()

In [10]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

---

In [12]:
# local 에서 작업할 꺼고 thread 는 2 개 사용할꺼고 app name 은 Spark Test 이다
sc = SparkContext(master="local[2]", appName="Spark Test")

In [13]:
sc

<SparkContext master=local[2] appName=Spark Test>

In [9]:
sc.stop()

---

### Conf를 통해 master driver를 생성하는 법 

In [16]:
from pyspark import SparkConf

In [17]:
conf = SparkConf()

In [21]:
conf = conf.setMaster("local[2]").setAppName("Conf")
conf

In [22]:
sc = SparkContext(conf=conf)

In [23]:
sc

<SparkContext master=local[2] appName=Conf>

---

## RDD
- Immutable --> read only --> falut-tolerance(Lineage)
- 데이터가 다 들어가 있고, 실제 데이터는 partition으로 조각조각 들어가 있음
- Transformation : map 
- Actions : reduce 

#### RDD 생성 : parallelize, textFile
#### RDD 확인 : glom, collect


In [14]:
# rdd 객체 가져옴 
rdd = sc.parallelize([1,2,3,4,5])

In [15]:
rdd 

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:184

In [16]:
rdd.collect()

[1, 2, 3, 4, 5]

In [17]:
# partition 마다 작업 : glom
rdd.glom().collect()

[[1, 2], [3, 4, 5]]

In [18]:
# 파일이 2개로 나눠서 작업되고 있음을 나타냄 
len(rdd.glom().collect())

2

In [19]:
# partition 3 개로 나눠서 작업할래 
rdd = sc.parallelize([1,2,3,4,5], 3)

In [20]:
# 3개인거 확인! 
rdd.glom().collect()

[[1], [2, 3], [4, 5]]

In [21]:
text = sc.parallelize("data/text-test.txt")

In [22]:
text.glom().collect()

[['d', 'a', 't', 'a', '/', 't', 'e', 'x', 't'],
 ['-', 't', 'e', 's', 't', '.', 't', 'x', 't']]

---

### Transformaitons
- map, filter, flatMap, distinct, sample, leftOuterJoin, rightOuterJoin, join, intersection, repartition

#### Map
- Return a new RDD by applying a function to each element of the RDD
- f : function 필요

In [23]:
rdd = sc.parallelize(["b", "a", "c"], 3)

In [24]:
rddMap = rdd.map(lambda x:(x,1))

In [25]:
rddMap

PythonRDD[8] at RDD at PythonRDD.scala:49

In [26]:
sorted(rddMap.glom().collect())

[[('a', 1)], [('b', 1)], [('c', 1)]]

#### Filter  

In [27]:
rdd = sc.parallelize([1,2,3,4,5])

In [28]:
rdd.filter(lambda x: x % 2 == 0).collect()

[2, 4]

#### FlatMap
- map과 유사하지만 리스트가 아닌 평면화된 결과
- 특정 컬럼에 무슨데이터가 있는지 모를때 사용
- 입력 데이터를 파싱할 때 올바르지 않은 형태의 데이터를 제거하기 위해 사용
- EX) 공란, 엉뚱한 값 in '성별' --> 정제해서 가져와야함 --> 중복되지 않는 값들 가져와야함

In [29]:
rdd = sc.parallelize([2,3,4])
rdd1 = rdd.map(lambda x:range(1,x))

In [30]:
rdd1.glom().collect()

[[range(1, 2)], [range(1, 3), range(1, 4)]]

In [31]:
# 평면화된 결과로 하나의 list로 가져옴
# range(1,2) :1 , range(1,3):1,2 , range(1:4) : 1,2,3
rdd.flatMap(lambda x:range(1,x)).collect()

[1, 1, 2, 1, 2, 3]

In [32]:
rdd = sc.parallelize(["Roes are red", "Violets are blue"])

In [33]:
rdd.map(lambda x: x.split()).collect()

[['Roes', 'are', 'red'], ['Violets', 'are', 'blue']]

In [34]:
rdd.flatMap(lambda x:x.split()).collect()

['Roes', 'are', 'red', 'Violets', 'are', 'blue']

#### distinct
- 중복된 결과를 없앤 값 리턴

In [35]:
sc.parallelize([1,1,2,3]).distinct().collect()

[2, 1, 3]

In [36]:
rdd.flatMap(lambda x:x.split()).distinct().collect()

['are', 'Roes', 'red', 'Violets', 'blue']

#### Sample
- return subset
- param 
    * withReplacement : 중복 허용 여부
    * fraction : 비율(얼마나 가져올꺼냐)
    * random seed number 

In [37]:
rdd = sc.parallelize(range(100), 4)
subset = rdd.sample(False, 0.1, 81)

In [38]:
subset

PythonRDD[30] at RDD at PythonRDD.scala:49

In [39]:
subset.collect()

[4, 26, 39, 41, 42, 52, 63, 76, 80, 86, 97]

In [40]:
subset.glom().collect()

[[4], [26, 39, 41, 42], [52, 63], [76, 80, 86, 97]]

In [41]:
# count() : spark 내부 함수, rdd 돌면서 개수셈 , 속도면에서 len 보다 빠름 
# 11개도 약 rane(100) 에서 0.1 비율
subset.count()

11

In [58]:
# python 내부 함수 
len(subset.collect())

11

#### JOIN 
- 두 데이터 셋을 섞어서 하나의 set 으로 보여지길 원함 

In [42]:
x = sc.parallelize([("a",1),("b",4)])
y = sc.parallelize([("a",2)])

In [43]:
x.glom().collect()

[[('a', 1)], [('b', 4)]]

In [85]:
#left join
# x 의 key 값을 기준으로해서 y 들어오는 값 join
sorted(x.leftOuterJoin(y).collect())

[('a', (1, 2)), ('b', (4, None))]

In [71]:
sorted(y.leftOuterJoin(x).collect())

[('a', (2, 1))]

In [72]:
sorted(x.rightOuterJoin(y).collect())

[('a', (1, 2))]

In [44]:
x = sc.parallelize([("a",1),("b",4)])
y = sc.parallelize([("a",2),("a",3)])

In [45]:
# 그냥 join 함수 : 두 RDD 의 교집합 키에 대한 값 
sorted(x.join(y).collect())

[('a', (1, 2)), ('a', (1, 3))]

In [47]:
sorted(y.join(x).collect())

[('a', (2, 1)), ('a', (3, 1))]

In [48]:
# intersection : 두 RDD 의 교집합 
# 완벽하게 일치하는게 없음 
sorted(x.intersection(y).collect())

[]

#### repartition 
- 데이터셋 재파티션

In [49]:
rdd = sc.parallelize([1,2,3,4,5,6,7], 4)

In [50]:
sorted(rdd.glom().collect())

[[1], [2, 3], [4, 5], [6, 7]]

In [51]:
rdd.repartition(10).glom().collect()

[[], [1], [4, 5, 6, 7], [2, 3], [], [], [], [], [], []]

In [79]:
sc.stop()

---

### Cache
- 자주 작업하는 것들은 cache를 잡아놓으면 좋음
- 사용자가 풀지 않으면 메모리를 계속 잡고 있다 

### Action
- collect, take, takeSample, reduce, reduceByKey
- count, countByKey, countByValue, foreach, foreachPartition
- saveAsTextFile

#### reduce 
- reduce : 특정함수를 사용해 RDD의 개수를 줄인다. key-value 쌍으로 된 것 
    - 1+ 2 = 3
    - 3 +4  = 7
    - 7 + 5 = 12
    - 3 + 12 = 15

In [52]:
rdd = sc.parallelize([1,2,3,4,5], 2)

In [53]:
rdd.glom().collect()

[[1, 2], [3, 4, 5]]

In [87]:
rdd.reduce(lambda x,y:x+y)

15

In [54]:
rdd = sc.parallelize([1,2,.5,.1,5,.2], 2)

In [55]:
rdd.glom().collect()

[[1, 2, 0.5], [0.1, 5, 0.2]]

In [91]:
# [1 , 0.1] --> 10.0
rdd.reduce(lambda x,y:x/y)

10.0

In [56]:
rdd = sc.parallelize([1,2,.5,.1,5,.2], 3)

In [57]:
rdd.glom().collect()

[[1, 2], [0.5, 0.1], [5, 0.2]]

In [93]:
# 0.5 , 5, 25 --> 0.1, 25 --> 0.004
rdd.reduce(lambda x,y:x/y)

0.004

In [58]:
rdd = sc.parallelize([('a',1),('b',2),('a',3)])

In [59]:
# 단어 key 값에 따른 value
from operator import add
sorted(rdd.reduceByKey(add).collect())

[('a', 4), ('b', 2)]

#### saveAsTextFile
- RDD 를 텍스트 파일로 저장 
- 모든 행이 문자열로 표현

In [62]:
rdd.glom().collect()

[[('a', 1)], [('b', 2), ('a', 3)]]

In [61]:
# 문서도 partition 으로 저장됨 
rdd.saveAsTextFile('data/abc')

#### textFile
- 파일로부터 데이터 읽기

In [63]:
fromText = sc.textFile('data/abc')

In [64]:
fromText.collect()

["('a', 1)", "('b', 2)", "('a', 3)"]

In [65]:
fromText.glom().collect()

[["('a', 1)"], ["('b', 2)", "('a', 3)"], []]

In [71]:
# minPartitions 가 5, 즉 최소 partition 이 5
fromText = sc.textFile('data/abc', minPartitions=5)

In [72]:
fromText.glom().collect()

[["('a', 1)"], [], ["('b', 2)"], ["('a', 3)"], [], []]

In [74]:
len(fromText.glom().collect())

6

---

In [76]:
# RDD 생성
data = sc.parallelize(
    [('a',22), {'b':23}, ['c',4]]
)

In [77]:
obj = data.collect()
type(obj)

list

In [78]:
obj[1]

{'b': 23}

In [79]:
obj[1]["b"]

23

---

In [81]:
from pyspark.sql import SparkSession

In [82]:
spark = SparkSession.builder.getOrCreate()

In [83]:
#driver 는 하나로 씀 
# 위에 SparkContext로 driver 하나 잡음.
# SparkSession.builder.getOrCreate() 로 driver 가 없으면 생성하고 있으면 잡아오기 때문에 에러가 나지 않고
# sparkContext 로 생성한 sc 가져옴. 
spark

In [84]:
sc.stop()

In [85]:
spark = SparkSession.builder.master("local[2]").appName("Spark Session").getOrCreate()

In [86]:
spark